Used for Folder Structure


In [ ]:
import os 
from PIL import Image
import shutil
import random

Used for DL and Visualisation

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

import torchvision
from torchvision import transforms

from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc, accuracy_score

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


# Folder Structure

Study of Image type and folder structure for further analysis

In [ ]:
# The folder is contained in this directory
animal_dir = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"

In [ ]:

len(os.listdir(animal_dir))

In [ ]:
with open("/kaggle/input/animal-image-dataset-90-different-animals/name of the animals.txt", 'r') as f:
    animal_info = f.read()

In [ ]:
print(animal_info.split())

In [ ]:
len(animal_info.split())

In [ ]:

animal_names = {} 
animal_directories = os.listdir(animal_dir)

for animal_name in animal_directories:
    animal_path = os.path.join(animal_dir, animal_name)
    num_images = len(os.listdir(animal_path))
    animal_names[animal_name] = num_images

In [ ]:
animal_names

In [ ]:

image_path = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals/antelope/02f4b3be2d.jpg"
image = Image.open(image_path)
width, height = image.size
print(f"width: {width}, height: {height}")

# **One vs Rest Classification**

This dataset class retrieves all images from the folder at the `ith` index, as well as `10%` of the images from each of the other folders. This approach helps to conserve memory and end potential train and test problems arising from significant discrepancies in image numbers across folders.


```
Note: Earlier the dataset took `ith` folder (60) images, and (5340) images in the rest folder, but this caused errors so, in the final version of the code this issue was solved using this method.
```

In [ ]:
path = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"

In the transform, the mean is ```mean=[0.485, 0.456, 0.406]``` and ```std=[0.229, 0.224, 0.225]``` to normalize images because it is given by PyTorch by default and is the Imagenet default values and in practice it is assumed to work better then rest values.

The label is 0 and 1.

In [ ]:
class dataset_new(torch.utils.data.Dataset):
    def __init__(self,data_idx,take=0.1,path=path):
        super().__init__()
        classes = os.listdir(path)
        folder_name = classes[data_idx]
        self.images = []
        self.labels = []
        self.transforms = transforms.Compose([
            transforms.Resize(256), # 256 for efficient net , 224
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        for folder in classes:
            if folder==folder_name:
                limit = 1
            else:
                limit = take
            anim_fold = os.path.join(path, folder)
            for l,localpath in enumerate(os.listdir(anim_fold)):
                if(l>=limit*len(os.listdir(anim_fold))):
                    break
                img_path = os.path.join(anim_fold, localpath)
                self.images.append(img_path)
                if folder==folder_name:
                    self.labels.append(1)
                else:
                    self.labels.append(0)
        
    def __len__(self):
        return len(self.images)
    def __getitem__(self,idx):
        img = Image.open(self.images[idx])
        label = self.labels[idx]
        return self.transforms(img), torch.tensor(label,dtype=torch.long)
      

```run_label_classification_one_vs_rest_fold```, this function is the main structure, and works to run the model using different models and perform calcualtions. It does 3fold CV with 3 epochs each using one_vs_rest and displays the test and train, accuracy and loss value after each epoch and also saves the plot.

In [ ]:
def run_label_classification_one_vs_rest_fold(model_net, name):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    name_model = str(name)
    for i in range(90):
        print(animal_info.split()[i])
        dataset = dataset_new(i)

        kfold = KFold(n_splits=3, shuffle=True)

        print('--------------------------------')

        for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

            print(f'FOLD {fold}')
            print('--------------------------------')

            train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
            test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

            train_loader = DataLoader(dataset, batch_size=32, sampler=train_subsampler, pin_memory=True, num_workers=2)
            test_loader = DataLoader(dataset, batch_size=32, sampler=test_subsampler, pin_memory=True, num_workers=2)

            model = model_net
            model.to(device)

            optimizer = optim.Adam(model.parameters(), lr=5e-5)
            train_losses = []
            test_losses = []
            train_accuracies = []
            test_accuracies = []

            for epoch in range(3):

                model.train()
                correct = 0
                total = 0
                train_loss = 0.0
                for _, (inputs, labels) in enumerate(train_loader):
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    outputs = model(inputs)
                    loss = nn.CrossEntropyLoss()(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    _, predicted = outputs.max(1)
                    total += labels.size(0)
                    correct += predicted.eq(labels).sum().item()

                model.eval()  
                test_loss = 0.0
                test_correct = 0
                test_total = 0
                all_labels = []
                all_predictions = []

                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = nn.CrossEntropyLoss()(outputs, labels)
                    test_loss += loss.item()*inputs.size(0)
                    _, predicted = outputs.max(1)
                    test_total += labels.size(0)
                    test_correct += predicted.eq(labels).sum().item()
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())


                test_loss = test_loss / len(test_loader)
                train_loss = train_loss/len(train_loader)
                test_accuracy = 100.0 * correct / total
                train_accuracy = 100.0 * test_correct/test_total

                train_losses.append(train_loss)
                test_losses.append(test_loss)
                train_accuracies.append(train_accuracy)
                test_accuracies.append(test_accuracy)

                print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f} \tTrain Accuracy {:.6f}% \tTest Accuracy: {:.2f}%'.format(
                    epoch+1, 
                    train_loss,
                    test_loss,
                    train_accuracy,
                    test_accuracy
                    ))
                
            directory = f'/kaggle/working/one-vs-rest/{name}/{animal_info.split()[i]}/'
            os.makedirs(directory, exist_ok=True)

            plt.figure(figsize=(10, 5))
            plt.plot(train_accuracies, label='Train Accuracy')
            plt.plot(test_accuracies, label='Test Accuracy')
            plt.title('Accuracy Curve for Dataset Name: {} Fold {}'.format(animal_info.split()[i], fold))
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.savefig(os.path.join(directory, f'{fold}.png'))


            print('Confusion Matrix for fold {}'.format(fold))
            print(confusion_matrix(all_labels, all_predictions))

Model using Efficientnet b0

In [ ]:
!pip install efficientnet_pytorch

In [ ]:

from efficientnet_pytorch import EfficientNet

class CEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.fc = nn.Linear(1000, 2)

    def forward(self, x):
        x = self.efficientnet(x)
        x = self.fc(x)
        return x

model_EfficientNet = CEfficientNet()
run_label_classification_one_vs_rest_fold(model_EfficientNet, 'Effnet')

Model using MobileNet

In [ ]:

from torchvision import models

class CMobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = models.mobilenet_v2(pretrained=True)
        self.fc = nn.Linear(1000, 2)

    def forward(self, x):
        x = self.mobilenet(x)
        x = self.fc(x)
        return x
    
model_MobileNet = CMobileNet()
run_label_classification_one_vs_rest_fold(model_MobileNet, 'Mobilenet')

Model using CustomNet

In [ ]:

class Attention(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.Tanh(),
            nn.Linear(in_features, 1),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        attention_weights = self.attention(x)
        return attention_weights * x

class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64*56*56, 1000) 
        self.attention = Attention(1000)
        self.fc2 = nn.Linear(1000, 2)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc1(x))
        x = self.attention(x)
        x = self.fc2(x)
        return x

model_CustomCNN = CustomCNN()
run_label_classification_one_vs_rest_fold(model_CustomCNN, 'CustomCNN')

# 5 Class Classification using 3 Fold CV

In [ ]:
path = "/kaggle/input/animal-image-dataset-90-different-animals/animals/animals"

In [ ]:
class dataset_pent_class(torch.utils.data.Dataset):
    def __init__(self,start_idx, path=path, window_size= 5):
        super().__init__()
        classes = os.listdir(path)
        self.images = []
        self.labels = []
        self.transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])
        end_idx = start_idx*5 + window_size
        for i in range(start_idx*5, end_idx):
            folder_name = classes[i] 
            anim_fold = os.path.join(path, folder_name)
            for localpath in os.listdir(anim_fold):
                img_path = os.path.join(anim_fold, localpath)
                self.images.append(img_path)
                self.labels.append(i % window_size)  
        
    def __len__(self):
        return len(self.images)
    def __getitem__(self,idx):
        img = Image.open(self.images[idx])
        label = self.labels[idx]
        return self.transforms(img), torch.tensor(label, dtype=torch.long)
      

```run_label_classification_one_vs_rest_fold``` even though the method name is the same, this takes multiple inputs, and saves the model weights. These weights are then loaded to visualise the model view images after the full process.

In [ ]:
def run_label_classification_one_vs_rest_fold(model_net, name, start_idx=0, window_size=5):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    name_model = str(name)
    for i in range(start_idx, 17, window_size):
        print(i, "structure folder")
        dataset = dataset_pent_class(i, window_size=window_size)

        kfold = KFold(n_splits=3, shuffle=True)

        print('--------------------------------')

        for fold, (train_ids, test_ids) in enumerate(kfold.split(dataset)):

            print(f'FOLD {fold}')
            print('--------------------------------')

            train_subsampler = torch.utils.data.SubsetRandomSampler(train_ids)
            test_subsampler = torch.utils.data.SubsetRandomSampler(test_ids)

            train_loader = DataLoader(dataset, batch_size=32, sampler=train_subsampler, pin_memory=True, num_workers=2)
            test_loader = DataLoader(dataset, batch_size=32, sampler=test_subsampler, pin_memory=True, num_workers=2)

            model = model_net
            model.to(device)

            optimizer = optim.Adam(model.parameters(), lr=5e-5)
            train_losses = []
            test_losses = []
            train_accuracies = []
            test_accuracies = []

            for epoch in range(3):

                model.train()
                correct = 0
                total = 0
                train_loss = 0.0
                for _, (inputs, labels) in enumerate(train_loader):
                    inputs, labels = inputs.to(device), labels.to(device)
                    optimizer.zero_grad()

                    outputs = model(inputs)
                    loss = nn.CrossEntropyLoss()(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    train_loss += loss.item()
                    _, predicted = outputs.max(1)
                    total += labels.size(0)
                    correct += predicted.eq(labels).sum().item()

                model.eval()  
                test_loss = 0.0
                test_correct = 0
                test_total = 0
                all_labels = []
                all_predictions = []

                for inputs, labels in test_loader:
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    loss = nn.CrossEntropyLoss()(outputs, labels)
                    test_loss += loss.item()*inputs.size(0)
                    _, predicted = outputs.max(1)
                    test_total += labels.size(0)
                    test_correct += predicted.eq(labels).sum().item()
                    all_labels.extend(labels.cpu().numpy())
                    all_predictions.extend(predicted.cpu().numpy())


                test_loss = test_loss / len(test_loader)
                train_loss = train_loss/len(train_loader)
                test_accuracy = 100.0 * correct / total
                train_accuracy = 100.0 * test_correct/test_total

                train_losses.append(train_loss)
                test_losses.append(test_loss)
                train_accuracies.append(train_accuracy)
                test_accuracies.append(test_accuracy)

                print('Epoch: {} \tTraining Loss: {:.6f} \tTest Loss: {:.6f} \tTrain Accuracy {:.6f}% \tTest Accuracy: {:.2f}%'.format(
                    epoch+1, 
                    train_loss,
                    test_loss,
                    train_accuracy,
                    test_accuracy
                    ))
                
            directory = f'/kaggle/working/5fold/{i}/{fold}/'
            os.makedirs(directory, exist_ok=True)

            plt.figure(figsize=(10, 5))
            plt.plot(train_accuracies, label='Train Accuracy')
            plt.plot(test_accuracies, label='Test Accuracy')
            plt.title('Accuracy Curve for {} Dataset, Fold {}'.format(i, fold))
            plt.xlabel('Epoch')
            plt.ylabel('Accuracy')
            plt.savefig(os.path.join(directory, f'{fold}.png'))
            
            model_directory = f'/kaggle/working/5fold_model/'
            os.makedirs(model_directory, exist_ok=True)
            torch.save(model.state_dict(), os.path.join(model_directory, f'{name_model}.pth'))


            print('Confusion Matrix for fold {}'.format(fold))
            print(confusion_matrix(all_labels, all_predictions))


EfficientNet b0


In [ ]:
!pip install efficientnet_pytorch

In [ ]:

from efficientnet_pytorch import EfficientNet

class CEfficientNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.efficientnet = EfficientNet.from_pretrained('efficientnet-b0')
        self.fc = nn.Linear(1000, 5)

    def forward(self, x):
        x = self.efficientnet(x)
        x = self.fc(x)
        return x

model_EfficientNet = CEfficientNet()
run_label_classification_one_vs_rest_fold(model_EfficientNet, 'EffNet', 0, 5)

MobileNet

In [ ]:

from torchvision import models

class CMobileNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.mobilenet = models.mobilenet_v2(pretrained=True)
        self.fc = nn.Linear(1000, 5)

    def forward(self, x):
        x = self.mobilenet(x)
        x = self.fc(x)
        return x
    
model_MobileNet = CMobileNet()
run_label_classification_one_vs_rest_fold(model_MobileNet, 'MobileNet', 0, 5)

Custom NeuralNetwork

In [ ]:


class Attention(nn.Module):
    def __init__(self, in_features):
        super().__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_features, in_features),
            nn.Tanh(),
            nn.Linear(in_features, 1),
            nn.Softmax(dim=1)
        )

    def forward(self, x):
        attention_weights = self.attention(x)
        return attention_weights * x
    
class CustomCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.fc1 = nn.Linear(64*56*56, 1000) 
        self.attention = Attention(1000)
        self.fc2 = nn.Linear(1000, 5) 

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, 2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, 2)
        x = x.view(x.size(0), -1) 
        x = F.relu(self.fc1(x))
        x = self.attention(x)
        x = self.fc2(x)
        return x

model_CustomCNN = CustomCNN()
run_label_classification_one_vs_rest_fold(model_CustomCNN, 'CustomCNN', 0, 5)

# Model Visualisation 

In [ ]:
import torchvision
from torchvision import models, transforms, utils
from torch.autograd import Variable
import scipy.misc
import json

In [ ]:
%matplotlib inline

Using the same transform as used during dataloading

Plotting the output of all convolutional layers and discuss the insights on automatically created features.

In [ ]:
#Taking an Image
image = Image.open(str('/kaggle/input/animal-image-dataset-90-different-animals/animals/animals/antelope/02f4b3be2d.jpg'))
plt.imshow(image)


# Function for Visual


In [ ]:
transforms = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

def nn_layer_visual(image, model, model_path):
        
        model.load_state_dict(torch.load(model_path))

        
        model_weights =[]
        conv_layers = []
        model_children = list(model.children())
        counter = 0
        for i in range(len(model_children)):
            if type(model_children[i]) == nn.Conv2d:
                counter+=1
                model_weights.append(model_children[i].weight)
                conv_layers.append(model_children[i])
            elif type(model_children[i]) == nn.Sequential:
                for j in range(len(model_children[i])):
                    for child in model_children[i][j].children():
                        if type(child) == nn.Conv2d:
                            counter+=1
                            model_weights.append(child.weight)
                            conv_layers.append(child)
        print(f"Total convolution layers: {counter}")
        print("conv_layers")

        image = transforms(image)
        print(f"Image shape before: {image.shape}")
        image = image.unsqueeze(0)
        print(f"Image shape after: {image.shape}")

        if len(image.shape) == 3:
            image = image.unsqueeze(0)

        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        image = image.to(device)

        print(f"Image shape before: {image.shape}")

        outputs = []
        names = []
        for layer in conv_layers[0:]:
            image = layer(image)
            outputs.append(image)
            names.append(str(layer))

        print(len(outputs))
        for feature_map in outputs:
            print(feature_map.shape)

        processed = []
        for feature_map in outputs:
            feature_map = feature_map.squeeze(0)
            gray_scale = torch.sum(feature_map,0)
            gray_scale = gray_scale / feature_map.shape[0]
            processed.append(gray_scale.data.cpu().numpy())
        for fm in processed:
            print(fm.shape)

        fig = plt.figure(figsize=(30, 50))
        for i in range(len(processed)):
            a = fig.add_subplot(5, 4, i+1)
            imgplot = plt.imshow(processed[i])
            a.axis("off")
            a.set_title(names[i].split('(')[0], fontsize=30)


# Custom CNN Visual

In [ ]:
nn_layer_visual(image, model_CustomCNN, '/kaggle/working/5fold_model/CustomCNN.pth')